In [98]:
import os
import numpy
import pandas as pd
from sqlalchemy import create_engine

In [99]:
host_name = "localhost"
port = "3306"
user_id = "root"
pwd = "Passw0rd123"

src_dbname = "chinook"
dst_dbname = "music_data"

src_engine = create_engine(f'mysql+mysqlconnector://{user_id}:{pwd}@{host_name}/{src_dbname}')
engine = create_engine(f'mysql+mysqlconnector://{user_id}:{pwd}@{host_name}/{dst_dbname}')

In [100]:
def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

In [101]:
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)
connection = sqlEngine.connect()

# The ETL Pipeline starts in the file fetching from the spotify api, and it already puts a tentative dim_tracks in music_data
# so we can't drop it and create it again here. Therefore these lines are commented out
#connection.execute(f"DROP DATABASE IF EXISTS `{dst_dbname}`;")
#connection.execute(f"CREATE DATABASE `{dst_dbname}`;")
connection.execute(f"USE {dst_dbname};")

connection.close()

In [102]:
# Load the song popularity data from csv file into a pandas df
song_pop_df = pd.read_csv('song_pop.csv', sep='\t')

song_pop_df.head(2)

,song_id,year_end_score,is_pop,year
0,0tgVpDi06FyKpA1z0VMD4v,5617,True,2018
1,3lFf4DnT68QjQGB3iq1BJ9,5549,True,2018


In [103]:
# Renaming columns to fit with conventions of rest of tables
song_pop_df.rename(columns={'year': 'MeasuredPopularityYear', 'is_pop': 'IsPopular', 'year_end_score': 'PopularityScore'}, inplace=True)
song_pop_df.head(2)

,song_id,PopularityScore,IsPopular,MeasuredPopularityYear
0,0tgVpDi06FyKpA1z0VMD4v,5617,True,2018
1,3lFf4DnT68QjQGB3iq1BJ9,5549,True,2018


In [104]:
# Import dim_tracks from dst database as a df
dim_tracks_df = pd.read_sql_table('dim_tracks', con=engine)
# Merge this with the song popularity dataframe
merged_df = pd.merge(dim_tracks_df, song_pop_df, left_on='spotify_track_id', right_on='song_id', how='left')

In [105]:
merged_df.head(100)

,spotify_track_id,song_name,artist_id,artist_name,date,track_number,duration_ms,album_id,album_name,track_count,song_id,PopularityScore,IsPopular,MeasuredPopularityYear
0,17lu4tymfnhmcIDlzBbtAb,For Those About To Rock (We Salute You),711MCceyCBcFnzjGY4Q7Un,AC/DC,1981-11-23,1,344240,7DUvURQ0wfA1kgG8j99frR,For Those About to Rock (We Salute You),10,NaN,NaN,NaN,NaN
1,2dbOYFfS7r7NAzquRvji9A,Balls to the Wall,3JDIAtVrJdQ7GFOX26LYpv,Accept,1983,1,342600,2twCPCDGJjVD90GWUjA8vN,Balls To The Wall,12,NaN,NaN,NaN,NaN
2,204523f9MSgEfc0oDJycNS,Fast As a Shark,3JDIAtVrJdQ7GFOX26LYpv,Accept,1982,1,228666,0xym0raBJhG64y1Oc179TX,Restless And Wild,10,NaN,NaN,NaN,NaN
3,5O0FdpixX4OhV75VVUEt2p,Restless and Wild,3JDIAtVrJdQ7GFOX26LYpv,Accept,1982,2,252093,0xym0raBJhG64y1Oc179TX,Restless And Wild,10,NaN,NaN,NaN,NaN
4,2CgXyaryyuxl35iMlvanD8,Princess of the Dawn,3JDIAtVrJdQ7GFOX26LYpv,Accept,1982,10,375466,0xym0raBJhG64y1Oc179TX,Restless And Wild,10,NaN,NaN,NaN,NaN
5,7tPPgAFhBC0EQqyUmREjoU,Put The Finger On You,5ShmqyBAJRFMF0PqVjLYF3,Parachute Express,1998-09-30,3,167106,2uSEzsfvDuz7ZVTI1ipyaG,Feel the Music,12,NaN,NaN,NaN,NaN
6,3D3xCU3Q6oKbvnfzMNPJki,Let's Get It Up,58AqJv0sDFPFnnBcLT8eeX,Bella Thorne,2014-01-01,10,150360,3cLSwFnXjZGfgM62w7BCuT,Disney Channel Play It Loud,14,NaN,NaN,NaN,NaN
7,31KSQBbHS275MsHgHmTD21,Inject The Venom,711MCceyCBcFnzjGY4Q7Un,AC/DC,1981-11-23,4,210893,7DUvURQ0wfA1kgG8j99frR,For Those About to Rock (We Salute You),10,NaN,NaN,NaN,NaN
8,1is13OusVjtwAvLx5PcxcZ,Snowballed,711MCceyCBcFnzjGY4Q7Un,AC/DC,1981-11-23,5,203133,7DUvURQ0wfA1kgG8j99frR,For Those About to Rock (We Salute You),10,NaN,NaN,NaN,NaN
9,5ahPl557niKxia8Kquw1yh,Evil Walks,711MCceyCBcFnzjGY4Q7Un,AC/DC,1981-11-23,6,263666,7DUvURQ0wfA1kgG8j99frR,For Those About to Rock (We Salute You),10,NaN,NaN,NaN,NaN


In [106]:
# Dropping unnecessary columns
merged_df.drop(['artist_id'], axis=1, inplace=True)
merged_df.drop(['album_id'], axis=1, inplace=True)
merged_df.drop(['track_count'], axis=1, inplace=True)
merged_df.drop(['song_id'], axis=1, inplace=True)

merged_df.head(100)

,spotify_track_id,song_name,artist_name,date,track_number,duration_ms,album_name,PopularityScore,IsPopular,MeasuredPopularityYear
0,17lu4tymfnhmcIDlzBbtAb,For Those About To Rock (We Salute You),AC/DC,1981-11-23,1,344240,For Those About to Rock (We Salute You),NaN,NaN,NaN
1,2dbOYFfS7r7NAzquRvji9A,Balls to the Wall,Accept,1983,1,342600,Balls To The Wall,NaN,NaN,NaN
2,204523f9MSgEfc0oDJycNS,Fast As a Shark,Accept,1982,1,228666,Restless And Wild,NaN,NaN,NaN
3,5O0FdpixX4OhV75VVUEt2p,Restless and Wild,Accept,1982,2,252093,Restless And Wild,NaN,NaN,NaN
4,2CgXyaryyuxl35iMlvanD8,Princess of the Dawn,Accept,1982,10,375466,Restless And Wild,NaN,NaN,NaN
5,7tPPgAFhBC0EQqyUmREjoU,Put The Finger On You,Parachute Express,1998-09-30,3,167106,Feel the Music,NaN,NaN,NaN
6,3D3xCU3Q6oKbvnfzMNPJki,Let's Get It Up,Bella Thorne,2014-01-01,10,150360,Disney Channel Play It Loud,NaN,NaN,NaN
7,31KSQBbHS275MsHgHmTD21,Inject The Venom,AC/DC,1981-11-23,4,210893,For Those About to Rock (We Salute You),NaN,NaN,NaN
8,1is13OusVjtwAvLx5PcxcZ,Snowballed,AC/DC,1981-11-23,5,203133,For Those About to Rock (We Salute You),NaN,NaN,NaN
9,5ahPl557niKxia8Kquw1yh,Evil Walks,AC/DC,1981-11-23,6,263666,For Those About to Rock (We Salute You),NaN,NaN,NaN


In [107]:
# Collecting extra track info from the chinook db track table
sql_track = "SELECT TrackId, Name FROM chinook.track;"
# Turning this into a dataframe
df_track = get_dataframe(user_id, pwd, host_name, src_dbname, sql_track)

df_track.head(5)

,TrackId,Name
0,1,For Those About To Rock (We Salute You)
1,2,Balls to the Wall
2,3,Fast As a Shark
3,4,Restless and Wild
4,5,Princess of the Dawn


In [108]:
# Merging the already merged track df with the track df from chinook db to get original TrackId
result_dim_tracks_df = pd.merge(df_track, merged_df, left_on='Name', right_on='song_name', how='inner')

In [109]:
result_dim_tracks_df.head(100)

,TrackId,Name,spotify_track_id,song_name,artist_name,date,track_number,duration_ms,album_name,PopularityScore,IsPopular,MeasuredPopularityYear
0,1,For Those About To Rock (We Salute You),17lu4tymfnhmcIDlzBbtAb,For Those About To Rock (We Salute You),AC/DC,1981-11-23,1,344240,For Those About to Rock (We Salute You),NaN,NaN,NaN
1,2,Balls to the Wall,2dbOYFfS7r7NAzquRvji9A,Balls to the Wall,Accept,1983,1,342600,Balls To The Wall,NaN,NaN,NaN
2,3,Fast As a Shark,204523f9MSgEfc0oDJycNS,Fast As a Shark,Accept,1982,1,228666,Restless And Wild,NaN,NaN,NaN
3,4,Restless and Wild,5O0FdpixX4OhV75VVUEt2p,Restless and Wild,Accept,1982,2,252093,Restless And Wild,NaN,NaN,NaN
4,5,Princess of the Dawn,2CgXyaryyuxl35iMlvanD8,Princess of the Dawn,Accept,1982,10,375466,Restless And Wild,NaN,NaN,NaN
5,6,Put The Finger On You,7tPPgAFhBC0EQqyUmREjoU,Put The Finger On You,Parachute Express,1998-09-30,3,167106,Feel the Music,NaN,NaN,NaN
6,7,Let's Get It Up,3D3xCU3Q6oKbvnfzMNPJki,Let's Get It Up,Bella Thorne,2014-01-01,10,150360,Disney Channel Play It Loud,NaN,NaN,NaN
7,8,Inject The Venom,31KSQBbHS275MsHgHmTD21,Inject The Venom,AC/DC,1981-11-23,4,210893,For Those About to Rock (We Salute You),NaN,NaN,NaN
8,9,Snowballed,1is13OusVjtwAvLx5PcxcZ,Snowballed,AC/DC,1981-11-23,5,203133,For Those About to Rock (We Salute You),NaN,NaN,NaN
9,10,Evil Walks,5ahPl557niKxia8Kquw1yh,Evil Walks,AC/DC,1981-11-23,6,263666,For Those About to Rock (We Salute You),NaN,NaN,NaN


In [110]:
# Dropping unnecessary cols
result_dim_tracks_df.drop(['song_name'], axis=1, inplace=True)
result_dim_tracks_df.drop(['spotify_track_id'], axis=1, inplace=True)
result_dim_tracks_df.head(100)

,TrackId,Name,artist_name,date,track_number,duration_ms,album_name,PopularityScore,IsPopular,MeasuredPopularityYear
0,1,For Those About To Rock (We Salute You),AC/DC,1981-11-23,1,344240,For Those About to Rock (We Salute You),NaN,NaN,NaN
1,2,Balls to the Wall,Accept,1983,1,342600,Balls To The Wall,NaN,NaN,NaN
2,3,Fast As a Shark,Accept,1982,1,228666,Restless And Wild,NaN,NaN,NaN
3,4,Restless and Wild,Accept,1982,2,252093,Restless And Wild,NaN,NaN,NaN
4,5,Princess of the Dawn,Accept,1982,10,375466,Restless And Wild,NaN,NaN,NaN
5,6,Put The Finger On You,Parachute Express,1998-09-30,3,167106,Feel the Music,NaN,NaN,NaN
6,7,Let's Get It Up,Bella Thorne,2014-01-01,10,150360,Disney Channel Play It Loud,NaN,NaN,NaN
7,8,Inject The Venom,AC/DC,1981-11-23,4,210893,For Those About to Rock (We Salute You),NaN,NaN,NaN
8,9,Snowballed,AC/DC,1981-11-23,5,203133,For Those About to Rock (We Salute You),NaN,NaN,NaN
9,10,Evil Walks,AC/DC,1981-11-23,6,263666,For Those About to Rock (We Salute You),NaN,NaN,NaN


In [111]:
# Renaming columns to better fit conventions of rest of tables
result_dim_tracks_df.rename(columns={'date': 'ReleaseDate'}, inplace=True)
result_dim_tracks_df.rename(columns={'track_number': 'AlbumTrackNumber'}, inplace=True)
result_dim_tracks_df.rename(columns={'album_name': 'AlbumName'}, inplace=True)
result_dim_tracks_df.rename(columns={'artist_name': 'Artist'}, inplace=True)
result_dim_tracks_df.rename(columns={'duration_ms': 'DurationMsec'}, inplace=True)

In [112]:
# Getting invoiceline table data from chinook into df
invoice_line_df = pd.read_sql("SELECT * FROM InvoiceLine", src_engine)
# Getting invoice table data from chinook into db
invoice_df = pd.read_sql("SELECT * FROM Invoice", src_engine)
# Getting track table data from chinook into db
track_df = pd.read_sql("SELECT * FROM Track", src_engine)
# Getting artist table data from chinook into db
artist_df = pd.read_sql("SELECT * FROM Artist", src_engine)

In [113]:
dim_artists_df = artist_df[['ArtistId', 'Name']].copy()
dim_artists_df.rename(columns={'Name': 'ArtistName'}, inplace=True)

In [114]:
dim_artists_df.head(100)

,ArtistId,ArtistName
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains
...,...,...
95,96,Jota Quest
96,97,João Suplicy
97,98,Judas Priest
98,99,Legião Urbana


In [115]:
# Merging artists dimension table with one from chinook to get original ArtistId
dim_artists_merged_df = pd.merge(dim_tracks_df, dim_artists_df, left_on='artist_name', right_on='ArtistName', how='inner')
dim_artists_merged_df.head(50)

,spotify_track_id,song_name,artist_id,artist_name,date,track_number,duration_ms,album_id,album_name,track_count,ArtistId,ArtistName
0,17lu4tymfnhmcIDlzBbtAb,For Those About To Rock (We Salute You),711MCceyCBcFnzjGY4Q7Un,AC/DC,1981-11-23,1,344240,7DUvURQ0wfA1kgG8j99frR,For Those About to Rock (We Salute You),10,1,AC/DC
1,31KSQBbHS275MsHgHmTD21,Inject The Venom,711MCceyCBcFnzjGY4Q7Un,AC/DC,1981-11-23,4,210893,7DUvURQ0wfA1kgG8j99frR,For Those About to Rock (We Salute You),10,1,AC/DC
2,1is13OusVjtwAvLx5PcxcZ,Snowballed,711MCceyCBcFnzjGY4Q7Un,AC/DC,1981-11-23,5,203133,7DUvURQ0wfA1kgG8j99frR,For Those About to Rock (We Salute You),10,1,AC/DC
3,5ahPl557niKxia8Kquw1yh,Evil Walks,711MCceyCBcFnzjGY4Q7Un,AC/DC,1981-11-23,6,263666,7DUvURQ0wfA1kgG8j99frR,For Those About to Rock (We Salute You),10,1,AC/DC
4,4NQJn1aSV9r6UmYErgzS4s,Night Of The Long Knives,711MCceyCBcFnzjGY4Q7Un,AC/DC,1981-11-23,9,205760,7DUvURQ0wfA1kgG8j99frR,For Those About to Rock (We Salute You),10,1,AC/DC
5,753KutoAy00apPsplMRetG,Let There Be Rock,711MCceyCBcFnzjGY4Q7Un,AC/DC,1977-03-21,3,366346,3JwFNl80WWeXPiKEjIFbqa,Let There Be Rock,8,1,AC/DC
6,79KUOd1p6yNjh2Jkr9IQHk,Hell Ain't A Bad Place To Be,711MCceyCBcFnzjGY4Q7Un,AC/DC,1977-03-21,7,254120,3JwFNl80WWeXPiKEjIFbqa,Let There Be Rock,8,1,AC/DC
7,4wYq5wugZDzQiMZQYG4wVB,Whole Lotta Rosie,711MCceyCBcFnzjGY4Q7Un,AC/DC,1977-03-21,8,333800,3JwFNl80WWeXPiKEjIFbqa,Let There Be Rock,8,1,AC/DC
8,2dbOYFfS7r7NAzquRvji9A,Balls to the Wall,3JDIAtVrJdQ7GFOX26LYpv,Accept,1983,1,342600,2twCPCDGJjVD90GWUjA8vN,Balls To The Wall,12,2,Accept
9,204523f9MSgEfc0oDJycNS,Fast As a Shark,3JDIAtVrJdQ7GFOX26LYpv,Accept,1982,1,228666,0xym0raBJhG64y1Oc179TX,Restless And Wild,10,2,Accept


In [116]:
# Load the artist popularity data from csv file into a pandas df
artist_pop_df = pd.read_csv('artist_pop.csv', sep='\t')
# Rename cols to better fit conventions of other tables
artist_pop_df.rename(columns={'year': 'MeasuredPopularityYear', 'is_pop': 'IsPopular', 'year_end_score': 'PopularityScore'}, inplace=True)

artist_pop_df.head(3)

,artist_id,PopularityScore,IsPopular,MeasuredPopularityYear
0,3TVXtAsR1Inumwj472S9r4,115282,True,2018
1,0LyfQWJT6nXafLPZqxe9Of,78376,True,2018
2,6eUKZXaKkcviH0Ku9w2n3V,60969,True,2018


In [117]:
dim_artist_final_df = pd.merge(dim_artists_merged_df, artist_pop_df, on='artist_id', how='inner')


In [118]:
# Dropping unecessary cols
dim_artist_final_df.drop(['spotify_track_id'], axis=1, inplace=True)
dim_artist_final_df.drop(['song_name'], axis=1, inplace=True)
dim_artist_final_df.drop(['artist_id'], axis=1, inplace=True)
dim_artist_final_df.drop(['artist_name'], axis=1, inplace=True)
dim_artist_final_df.drop(['date'], axis=1, inplace=True)
dim_artist_final_df.drop(['track_number'], axis=1, inplace=True)
dim_artist_final_df.drop(['duration_ms'], axis=1, inplace=True)
dim_artist_final_df.drop(['album_id'], axis=1, inplace=True)
dim_artist_final_df.drop(['album_name'], axis=1, inplace=True)
dim_artist_final_df.drop(['track_count'], axis=1, inplace=True)

# Dropping duplicate rows
dim_artist_final_df = dim_artist_final_df.drop_duplicates()

dim_artist_final_df.head(100)

,ArtistId,ArtistName,PopularityScore,IsPopular,MeasuredPopularityYear
0,1,AC/DC,19627,True,2018
1,1,AC/DC,9272,True,2017
2,1,AC/DC,17461,True,2016
3,1,AC/DC,22897,True,2015
4,1,AC/DC,16351,True,2014
...,...,...,...,...,...
336,3,Aerosmith,3691,True,1974
337,3,Aerosmith,664,True,1973
597,4,Alanis Morissette,1302,True,2016
598,4,Alanis Morissette,1605,True,2015


In [119]:
# Merging invoiceline df with selected columns from invoice df
fact_sales_basic = pd.merge(invoice_line_df, invoice_df[['InvoiceId', 'CustomerId', 'InvoiceDate']], on='InvoiceId')
# Merging the previously combined df fact_sales_basic with selected columns from track_df
fact_sales_df = pd.merge(fact_sales_basic, track_df[['TrackId', 'Name']], on='TrackId', how='left')
# Rename columns for clarity and consistency
fact_sales_df.rename(columns={'InvoiceDate': 'SaleDate', 'Name': 'TrackName'}, inplace=True)

In [120]:
# Merging fact sales df with dim tracks df for some extra info
fact_sales_df = pd.merge(fact_sales_df, result_dim_tracks_df, on='TrackId', how='left')


In [121]:
fact_sales_df.head(10)

,InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity,CustomerId,SaleDate,TrackName,Name,Artist,ReleaseDate,AlbumTrackNumber,DurationMsec,AlbumName,PopularityScore,IsPopular,MeasuredPopularityYear
0,1,1,2,0.99,1,2,2021-01-01,Balls to the Wall,Balls to the Wall,Accept,1983,1.0,342600.0,Balls To The Wall,NaN,NaN,NaN
1,2,1,4,0.99,1,2,2021-01-01,Restless and Wild,Restless and Wild,Accept,1982,2.0,252093.0,Restless And Wild,NaN,NaN,NaN
2,3,2,6,0.99,1,4,2021-01-02,Put The Finger On You,Put The Finger On You,Parachute Express,1998-09-30,3.0,167106.0,Feel the Music,NaN,NaN,NaN
3,4,2,8,0.99,1,4,2021-01-02,Inject The Venom,Inject The Venom,AC/DC,1981-11-23,4.0,210893.0,For Those About to Rock (We Salute You),NaN,NaN,NaN
4,5,2,10,0.99,1,4,2021-01-02,Evil Walks,Evil Walks,AC/DC,1981-11-23,6.0,263666.0,For Those About to Rock (We Salute You),NaN,NaN,NaN
5,6,2,12,0.99,1,4,2021-01-02,Breaking The Rules,Breaking The Rules,Atlus,2023-05-26,12.0,123637.0,High Expectations,NaN,NaN,NaN
6,7,3,16,0.99,1,8,2021-01-03,Dog Eat Dog,Dog Eat Dog,Tommy Lefroy,2022-07-27,1.0,169200.0,Dog Eat Dog,NaN,NaN,NaN
7,8,3,20,0.99,1,8,2021-01-03,Overdose,Overdose,honestav,2024-01-13,1.0,235885.0,I’d Rather Overdose,NaN,NaN,NaN
8,9,3,24,0.99,1,8,2021-01-03,Love In An Elevator,Love In An Elevator,Aerosmith,1994-01-01,2.0,321826.0,Big Ones,NaN,NaN,NaN
9,10,3,28,0.99,1,8,2021-01-03,Janie's Got A Gun,Janie's Got A Gun,Aerosmith,2011-01-01,11.0,328560.0,Tough Love: Best Of The Ballads,NaN,NaN,NaN


In [122]:
# Dropping unnecessary cols
fact_sales_df.drop(['Name'], axis=1, inplace=True)
fact_sales_df.drop(['MeasuredPopularityYear'], axis=1, inplace=True)
fact_sales_df.drop(['IsPopular'], axis=1, inplace=True)
fact_sales_df.drop(['PopularityScore'], axis=1, inplace=True)
fact_sales_df.drop(['AlbumTrackNumber'], axis=1, inplace=True)
fact_sales_df.drop(['DurationMsec'], axis=1, inplace=True)




In [123]:
fact_sales_df.head(100)

,InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity,CustomerId,SaleDate,TrackName,Artist,ReleaseDate,AlbumName
0,1,1,2,0.99,1,2,2021-01-01,Balls to the Wall,Accept,1983,Balls To The Wall
1,2,1,4,0.99,1,2,2021-01-01,Restless and Wild,Accept,1982,Restless And Wild
2,3,2,6,0.99,1,4,2021-01-02,Put The Finger On You,Parachute Express,1998-09-30,Feel the Music
3,4,2,8,0.99,1,4,2021-01-02,Inject The Venom,AC/DC,1981-11-23,For Those About to Rock (We Salute You)
4,5,2,10,0.99,1,4,2021-01-02,Evil Walks,AC/DC,1981-11-23,For Those About to Rock (We Salute You)
...,...,...,...,...,...,...,...,...,...,...,...
95,96,18,548,0.99,1,31,2021-03-09,Smoke On The Water,NaN,NaN,NaN
96,97,18,554,0.99,1,31,2021-03-09,Trampled Under Foot,NaN,NaN,NaN
97,98,19,563,0.99,1,40,2021-03-14,União Da Ilha,NaN,NaN,NaN
98,99,19,572,0.99,1,40,2021-03-14,Put Your Lights On,NaN,NaN,NaN


In [124]:
start_date = '2000-01-01'
end_date = '2024-12-31'

# Set date range for df
dates = pd.date_range(start=start_date, end=end_date)

In [125]:
# Create dim_date_df with a single column 'Date' containing date values
dim_date_df = pd.DataFrame({'Date': dates})

# Add a more columns to the DataFrame by extracting each from the 'Date' column
dim_date_df['Year'] = dim_date_df['Date'].dt.year
dim_date_df['Month'] = dim_date_df['Date'].dt.month
dim_date_df['Day'] = dim_date_df['Date'].dt.day
dim_date_df['WeekOfYear'] = dim_date_df['Date'].dt.isocalendar().week
dim_date_df['Quarter'] = dim_date_df['Date'].dt.quarter
dim_date_df['DayOfWeek'] = dim_date_df['Date'].dt.dayofweek
dim_date_df['DayName'] = dim_date_df['Date'].dt.day_name()
dim_date_df['IsWeekend'] = dim_date_df['DayOfWeek'].isin([5, 6]).astype(int)

In [126]:
# Sending all dfs to music_data db as tables
fact_sales_df.to_sql(name='fact_sales', con=engine, if_exists='replace', index=False)
dim_artist_final_df.to_sql(name='dim_artist', con=engine, if_exists='replace', index=False)
dim_date_df.to_sql(name='dim_date', con=engine, if_exists='replace', index=False)
result_dim_tracks_df.to_sql(name='dim_tracks', con=engine, if_exists='replace', index=False)

59